In [1]:
from thesis_initialise import *

In [2]:
from merge import *

In [3]:
# merge('../data')
# reader = Reader('merged', dataDir)
# inputs = get_inputs(reader)

In [19]:
isoinputs, arrinputs, plastinputs = (
    make_frame(get_inputs()[key])
        for key in {'Isovisc', 'Arrhenius', 'Viscoplastic'}
    )

In [20]:
isoinputs

,H,aspect,f,res,tauRef,temperatureField
ionguiseok-vuachlekr,0.0,1.0,0.5,64.0,7.943282e+04,_built_oiskeaosle-woatihoo
ifriiiomaa-pheescaosoe,0.0,1.0,0.5,64.0,8.912509e+04,_built_eaqaiblii-keeflkey
staskeugruu-kwiighseefl,0.0,1.0,0.5,64.0,8.912509e+04,_built_houtzghaar-bierfrout
vuotrspaj-tiapleblee,0.0,1.0,0.5,64.0,8.912509e+04,_built_iubliiubree-dwiokruzau
eaplatzium-swoishuyua,0.0,1.0,0.5,64.0,8.912509e+04,_built_oiskeaosle-woatihoo
...,...,...,...,...,...,...
brismaiceu-kuutsxip,10.0,1.414,1.0,64.0,7.079458e+05,_built_peaskauslu-thoesfthuec
ueluuokweu-iekweepseoh,10.0,1.414,1.0,64.0,7.943282e+05,_built_peaskauslu-thoesfthuec
peitseopluu-psuismzieth,10.0,1.414,1.0,64.0,8.912509e+05,_built_peaskauslu-thoesfthuec
otweoepsei-waaswngiesl,10.0,1.414,1.0,64.0,1.000000e+06,_built_peaskauslu-thoesfthuec


In [21]:
plastinputs

,H,alpha,aspect,f,res,temperatureField
aagriatseukw-raprzaitr,0.0,4.0,0.707000,0.5,16.0,_built_peaskauslu-thoesfthuec
smoiskloepl-drooscfiom,0.0,4.0,0.707000,0.6,16.0,_built_peaskauslu-thoesfthuec
euskoaucei-psieklfeips,0.0,4.0,0.707000,0.7,16.0,_built_peaskauslu-thoesfthuec
aedwuegifl-slishhipl,0.0,4.0,0.707000,0.8,16.0,_built_peaskauslu-thoesfthuec
eothuebluagh-uislugloej,0.0,4.0,0.707000,0.9,16.0,_built_peaskauslu-thoesfthuec
...,...,...,...,...,...,...
staugrbeuph-troetrswiarh,10.0,7.0,1.414214,0.6,96.0,_built_eafiseoch-aetwooeiyue
skuizraur-heigriuvau,10.0,7.0,1.414214,0.7,96.0,_built_kaatuubua-aufraeeuwue
uaraopsaag-eeghuetsuutz,10.0,7.0,1.414214,0.8,96.0,_built_aatzuuebloo-sciehxiux
iifuedroasw-waorhaoprau,10.0,7.0,1.414214,0.9,96.0,_built_snoiscbreudw-uipsouoiswau


In [22]:
import os

In [23]:
os.path.dirname('a/b/c')

'a/b'